# Building the Transformer from scratch

In this notebook, we'll be implementing the famous Transformer architecture from scratch

The code is based off of the following repos/blog posts:

<a href="http://nlp.seas.harvard.edu/annotated-transformer/">The Annotated Transformer</a>

<a href="https://arxiv.org/abs/1706.03762">Attention Is All You Need</a>

<a href="https://keremzaman.com/2020/10/12/role-of-the-target-mask-during-transformer-training/">Role of the Target Mask during Transformer Training</a>





In [1]:
import torch 
from torch import Tensor
import torch.nn as nn
import numpy as np

/home/yubaes/anaconda3/envs/DL/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


One of the keys to understanding how any model works is understanding how the shapes of the tensors change during the processing of each part. We'll be using the logging module to output debugging information to help our understanding.

In [2]:
import logging
logger = logging.getLogger('tensor_shapes')
handler = logging.StreamHandler()
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
# if you want the model to continuously print tensor shapes, set to DEBUG!
logger.setLevel(1)

In [3]:
import inspect
def getclass():
    stack = inspect.stack()
    return stack[3][0].f_locals["self"].__class__

# A helper function to check how tensor sizes change
def log_size(tsr: torch.Tensor, name: str):
    cls = getclass()
    logger.log(level=cls.level, msg=f"[{cls.__name__}] {name} size={tsr.shape}")

We'll use logging levels to control the modules we receive output from. The lower the logging level, the more tensor information you'll get. Feel free to play around!

In [4]:
from enum import IntEnum
# Control how much debugging output we want
class TensorLoggingLevels(IntEnum):
    attention = 1
    attention_head = 2
    multihead_attention_block = 3
    enc_dec_block = 4
    enc_dec = 5



We'll be using an enum to refer to dimensions whenever possible to improve readability.


In [5]:
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2


# Components

# Scaled dot product attention

The Transformer is an attention-based architecture. The attention used in the Transformer is the scaled dot product attention, represented by the following formula.
                
                
                attention(Q,K,V) = softmax(QK^T/sqrt(dk)V



In [6]:
import math

class ScaledDotProductAttention(nn.Module):
    level = TensorLoggingLevels.attention
    def __init__(self,dropout:float=0.1)->None:
        super().__init__()
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,q:Tensor,k:Tensor,v:Tensor,mask:Tensor=None)->Tensor:
        d_k = k.size(-1) # get the size of the key
        assert q.size(-1) == d_k
        
        # Compute the dot product between queries and keys for 
        # each batch and position in the sequence
        attn = torch.bmm(q,k.transpose(Dim.seq,Dim.feature)) # (batch_size,seq_len,seq_len)
        #we get an attention score between each position in the sequence for each batch
        
        # Scale the dot product by the dimentionality
        attn = attn/math.sqrt(d_k)
        
        # normalize the weights across the sequence dimension
        # (Note that since we transposed, the sequence and feature dimensions are switched)
        attn = torch.exp(attn)
        log_size(attn, "attention weight") # (batch_size, Seq_len, Seq_len)
        
        # fill attention weights with 0s where padded
        if mask is not None: 
            attn = attn.masked_fill(mask, 0)
        attn = attn / attn.sum(dim=-1, keepdim=True)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v) # (batch_size, seq_len, feature)
        log_size(output, "attention output size") # (batch_size, seq_len, seq_len)
        return output
        

In [7]:
attn = ScaledDotProductAttention()

In [8]:
q = torch.rand(5, 10, 20)
k = torch.rand(5, 10, 20)
v = torch.rand(5, 10, 20)

In [9]:
attn(q, k, v)

[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[0.6760, 0.4856, 0.4845, 0.5843, 0.4901, 0.6175, 0.8554, 0.6084,
          0.6465, 0.5246, 0.6200, 0.6058, 0.5926, 0.7605, 0.5624, 0.4588,
          0.4617, 0.5583, 0.5743, 0.6928],
         [0.5456, 0.3446, 0.3813, 0.3700, 0.3816, 0.4067, 0.5704, 0.3192,
          0.4979, 0.3760, 0.4142, 0.4648, 0.4197, 0.5194, 0.4566, 0.3177,
          0.3655, 0.3899, 0.4278, 0.5480],
         [0.4954, 0.2710, 0.3246, 0.4519, 0.3532, 0.5428, 0.6357, 0.5354,
          0.5623, 0.4407, 0.5359, 0.4190, 0.4410, 0.6435, 0.3971, 0.3051,
          0.3373, 0.4196, 0.4540, 0.5259],
         [0.6719, 0.4723, 0.4687, 0.5876, 0.4894, 0.5970, 0.8405, 0.6131,
          0.6415, 0.5361, 0.6324, 0.5991, 0.5875, 0.7587, 0.5704, 0.4445,
          0.4646, 0.5627, 0.5833, 0.6909],
         [0.5898, 0.3503, 0.4122, 0.4838, 0.4577, 0.4870, 0.7154, 0.5331,
          0.5651, 0.5132, 0.6121, 0.5129, 0.5074, 0.6940, 0.5629, 0.3917,
          0.4178, 0.4978, 0.4806, 0.6203],
         [0.6786, 0.4878, 0.4807, 0.5815, 0.4


# Multi-Head Attention

Now, we turn to the core component in the Transformer architecture: the multi-head attention block. This block applies linear transformations to the input, then applies scaled dot product attention.


In [13]:
class AttentionHead(nn.Module):
    level = TensorLoggingLevels.attention_head
    def __init__(self,d_model:int,d_feature:int,dropout:float=0.1)->None:
        super().__init__()
        
        self.attn = ScaledDotProductAttention(dropout)
        self.Q = nn.Linear(d_model,d_feature)
        self.K = nn.Linear(d_model,d_feature)
        self.V = nn.Linear(d_model,d_feature)
    
    def forward(self,q:Tensor,k:Tensor,v:Tensor,mask:Tensor=None)->Tensor:
        query = self.Q(q) # (batch_size,seq,feature)
        key = self.K(k)   # (batch_size,seq,feature)
        value = self.V(v) # (batch_size,seq,feature)
        log_size(query,"queries,keys,values")
        return self.attn(query,key,value)
        
        

In [14]:
attn_head = AttentionHead(20, 20)
attn_head(q, k, v)

[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[ 5.9640e-01,  5.2956e-01,  1.1770e-03,  7.5713e-02,  5.1480e-01,
           1.1813e-01,  7.2734e-02, -4.2564e-01, -1.9428e-01,  3.5302e-01,
          -3.1689e-01,  3.5237e-02, -6.9424e-01, -5.5099e-01,  4.2387e-01,
          -4.5039e-01, -2.7075e-01,  4.8245e-01,  5.0978e-01,  1.8877e-03],
         [ 5.9622e-01,  5.3147e-01,  6.4796e-04,  7.6590e-02,  5.1329e-01,
           1.1826e-01,  7.4331e-02, -4.2290e-01, -1.9048e-01,  3.5191e-01,
          -3.1679e-01,  3.4965e-02, -6.9292e-01, -5.5001e-01,  4.2325e-01,
          -4.5194e-01, -2.6899e-01,  4.8235e-01,  5.1077e-01, -2.4627e-04],
         [ 5.9682e-01,  5.3452e-01,  1.4155e-03,  7.5207e-02,  5.1304e-01,
           1.1663e-01,  7.3342e-02, -4.2723e-01, -1.8908e-01,  3.5151e-01,
          -3.1971e-01,  3.3672e-02, -6.9690e-01, -5.4693e-01,  4.2368e-01,
          -4.5345e-01, -2.7034e-01,  4.8447e-01,  5.1332e-01,  2.0362e-03],
         [ 5.1626e-01,  4.9480e-01,  2.1129e-02,  1.0486e-01,  4.7905e-01,
           7.8520e-02,

The multi-head attention block simply applies multiple attention heads, then concatenates the outputs and applies a single linear projection.

In [15]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.attention_head)

In [20]:
class MultiHeadAttention(nn.Module):
    level = TensorLoggingLevels.multihead_attention_block
    def __init__(self,d_model:int,d_feature:int,n_heads:int,dropout:float=0.1)->None:
        super().__init__()
        
        self.d_model = d_model
        self.d_feature = d_feature
        self.num_heads = n_heads
        
        assert d_model == d_feature*n_heads
        
        # Note that this is very inefficient:
        # I am merely implementing the heads separately because it is 
        # easier to understand this way
        
        self.attn_heads = nn.ModuleList([
            AttentionHead(d_model,d_feature,dropout) for _ in range(self.num_heads)
        ])
        
        self.projection = nn.Linear(d_feature*n_heads,d_model)
    
    def forward(self,q:Tensor,k:Tensor,v:Tensor,mask:Tensor=None)->Tensor:
        log_size(q,'input queries')
        x = [attn(q,k,v,mask=mask) for i,attn in enumerate(self.attn_heads)]
        log_size(x[0],'output of signle head')
        
        # reconcatenate
        x = torch.cat(x,dim=Dim.feature) #(batch_size,seq_len,d_feature*n_heads)
        log_size(x,'concatenated output')
        x = self.projection(x)#(batch_size,seq_len,d_feature)
        log_size(x,'projected output')
        return x
        
        

In [21]:
heads = MultiHeadAttention(20 * 8, 20, 8)
heads(q.repeat(1, 1, 8), 
      k.repeat(1, 1, 8), 
      v.repeat(1, 1, 8))

[MultiHeadAttention] input queries size=torch.Size([5, 10, 160])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[AttentionHead] queries,keys,values size=torch.Size([5, 10, 20])
[MultiHeadAttention] output of signle head size=torch.Size([5, 10, 20])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 160])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 160])


tensor([[[ 0.0511, -0.2162, -0.0687,  ..., -0.2284, -0.1313,  0.0634],
         [ 0.0507, -0.1710,  0.0064,  ..., -0.1789, -0.0826,  0.0415],
         [-0.0147, -0.1924, -0.0692,  ..., -0.1898, -0.1208,  0.0776],
         ...,
         [-0.0007, -0.1952, -0.0171,  ..., -0.1774, -0.1086,  0.0372],
         [-0.0367, -0.1986, -0.0429,  ..., -0.1955, -0.1282,  0.0520],
         [ 0.0745, -0.1993,  0.0189,  ..., -0.1806, -0.0612,  0.0701]],

        [[-0.0105, -0.1297, -0.1032,  ..., -0.1775, -0.1145,  0.1008],
         [ 0.0313, -0.1485, -0.0542,  ..., -0.1894, -0.1066,  0.0829],
         [ 0.0641, -0.1349, -0.0294,  ..., -0.1749, -0.1080,  0.0809],
         ...,
         [ 0.0311, -0.1679, -0.0134,  ..., -0.1700, -0.1206,  0.0759],
         [ 0.0374, -0.1427, -0.0724,  ..., -0.1909, -0.0974,  0.0973],
         [ 0.0078, -0.1118, -0.0805,  ..., -0.1774, -0.1151,  0.0668]],

        [[ 0.0048, -0.0978, -0.0691,  ..., -0.1836, -0.1724, -0.0024],
         [ 0.0533, -0.0943, -0.0861,  ..., -0


# The Encoder

With these core components in place, implementing the encoder is pretty easy.


The encoder consists of the following components:

    A multi-head attention block
    A simple feedforward neural network

These components are connected using residual connections and layer normalization


In [22]:
# We'll supress logging from the individual attention heads
logger.setLevel(TensorLoggingLevels.multihead_attention_block)


Layer normalization is similar to batch normalization, but normalizes across the feature dimension instead of the batch dimension.

In [26]:
class LayerNorm(nn.Module):
    def __init__(self,d_model:int,eps:float=1e-8)->None:
        
        super().__init__()
        
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps
    
    def forward(self,x:Tensor)->Tensor:
        mean = x.mean(-1,keepdim=True)
        std = x.std(-1,keepdim=True)
        return self.gamma * (x - mean)/(std + self.eps) + self.beta




The encoder just stacks these together


In [27]:
class EncoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self,d_model:int=512,d_feature:int=64,d_ff:int=2048,n_heads:int=8,dropout:float=0.1)->None:
        
        super().__init__()
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.layer_norm1 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        self.layer_norm2 = LayerNorm(d_model)
    
    def forward(self, x:Tensor, mask:Tensor=None)->Tensor:
        log_size(x, "Encoder block input")
        att = self.attn_head(x, x, x, mask=mask)
        log_size(x, "Attention output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm1(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        log_size(x, "Feedforward output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm2(pos))
        log_size(x, "Encoder size output")
        return x
        
    

In [28]:
enc = EncoderBlock()
enc(torch.rand(5, 10, 512))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of signle head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])


tensor([[[ 1.8192,  2.3789,  1.8456,  ...,  0.5327,  2.4587, -2.0722],
         [ 0.9339,  1.8649,  2.3383,  ..., -0.2168,  1.9650, -1.7625],
         [ 2.1261,  2.1873,  1.9888,  ...,  0.8242,  1.7350, -2.3299],
         ...,
         [ 1.4683,  0.3586,  0.9493,  ..., -1.1030,  2.2486, -1.2379],
         [ 1.7434,  2.4278,  1.8904,  ..., -0.5588,  1.8694, -1.2652],
         [ 1.6837,  1.2186,  1.6494,  ..., -0.1739,  1.1170, -2.0326]],

        [[ 1.9094,  1.3976,  2.1129,  ...,  0.5157,  2.7508, -1.8893],
         [ 2.0656,  1.6296,  2.4549,  ...,  0.0964,  3.1609, -1.7532],
         [-0.1060,  1.2345,  2.2978,  ...,  0.5943,  1.9315, -2.3323],
         ...,
         [ 1.0247,  2.5539,  1.6837,  ...,  0.5987,  2.9148, -1.8273],
         [ 1.7906,  2.0697,  1.0290,  ...,  1.0575,  2.3941, -2.4083],
         [ 1.7945,  1.1808,  1.7679,  ...,  0.5292,  2.0943, -2.7917]],

        [[ 1.1526,  2.7424,  2.4824,  ...,  0.8795,  1.2366, -1.9735],
         [ 0.6394,  2.2348,  1.7236,  ...,  0



The encoder consists of 6 consecutive encoder blocks, so can simply be implemented like the following


In [29]:
class TransformerEncoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self,n_blocks:int=6,d_model:int=512,n_heads:int=8,d_ff:int=2048,dropout:float=0.1)->None:
        super().__init__()
        self.encoders = nn.ModuleList([
            EncoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
    
    def forward(self, x: torch.FloatTensor, mask:Tensor=None)->Tensor:
        for encoder in self.encoders:
            x = encoder(x)
        return x
        
        


The Decoder

The decoder is mostly the same as the encoder. There's just one additional multi-head attention block that takes the target sentence as input.


The keys and values are the outputs of the encoder, and the queries are the outputs of the multi-head attention over the target entence embeddings.


In [32]:
class DecoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model:int=512, d_feature:int=64,
                 d_ff:int=2048, n_heads:int=8, dropout:float=0.1):
        super().__init__()
        self.masked_attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )

        self.layer_norm1 = LayerNorm(d_model)
        self.layer_norm2 = LayerNorm(d_model)
        self.layer_norm3 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x:Tensor, enc_out:Tensor, 
                src_mask:Tensor=None, tgt_mask:Tensor=None)->Tensor:
        # Apply attention to inputs
        att = self.masked_attn_head(x, x, x, mask=src_mask)
        x = x + self.dropout(self.layer_norm1(att))
        # Apply attention to the encoder outputs and outputs of the previous layer
        att = self.attn_head(x,enc_out,enc_out,tgt_mask)
        x = x + self.dropout(self.layer_norm2(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        x = x + self.dropout(self.layer_norm2(pos))
        return x

In [33]:
dec = DecoderBlock()
dec(torch.rand(5, 10, 512), enc(torch.rand(5, 10, 512)))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of signle head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])
[MultiHeadAttention] input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of signle head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[MultiHeadAttention] input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of signle head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Siz

tensor([[[-2.2615e+00, -1.8722e+00,  2.2510e+00,  ...,  2.9517e-01,
           9.4605e-01,  2.6810e-01],
         [-8.9229e-01, -3.7514e+00,  2.5797e+00,  ..., -3.6178e-01,
          -5.6768e-02,  1.4426e+00],
         [-7.3190e-01, -3.3193e+00,  2.3307e+00,  ...,  1.3170e-01,
           4.4112e-01,  7.1429e-01],
         ...,
         [-1.7197e+00, -1.1160e+00,  2.4876e+00,  ...,  2.5105e-01,
          -3.0039e-01, -3.1281e-01],
         [-2.4196e+00, -3.6260e+00,  2.2973e+00,  ..., -4.2077e-01,
           2.5133e+00, -6.5064e-01],
         [-1.6447e+00, -2.5834e+00,  1.2424e+00,  ...,  1.9222e-03,
          -4.5628e-01,  8.8122e-01]],

        [[-1.2983e+00, -9.8285e-01,  1.5088e+00,  ..., -4.7545e-01,
          -3.6731e-01, -5.1744e-01],
         [-1.3096e+00, -2.6271e+00,  3.9300e+00,  ...,  1.2478e+00,
           2.2432e-01, -8.9195e-01],
         [-1.9818e+00, -1.3137e+00,  3.6184e+00,  ..., -1.3409e+00,
           1.5128e+00,  6.6361e-01],
         ...,
         [-1.6676e+00, -3



Again, the decoder is just a stack of the underlying block so is simple to implement.


In [34]:
class TransformerDecoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.position_embedding = PositionalEmbedding(d_model)
        self.decoders = nn.ModuleList([
            DecoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
        
    def forward(self, x: torch.FloatTensor, 
                enc_out: torch.FloatTensor, 
                src_mask=None, tgt_mask=None):
        for decoder in self.decoders:
            x = decoder(x, enc_out, src_mask,tgt_mask)
        return x




# Positional Embeddings

Attention blocks are just simple matrix multiplications: therefore they don't have any notion of order! The Transformer explicitly adds positional information via the positional embeddings.


In [35]:
class PositionalEmbedding(nn.Module):
    level = 1
    def __init__(self, d_model, max_len=512):
        super().__init__()        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=False)
        
    def forward(self, x):
        return self.weight[:, :x.size(1), :] # (1, Seq, Feature)

In [36]:
class WordPositionEmbedding(nn.Module):
    level = 1
    def __init__(self, vocab_size, d_model=512):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = PositionalEmbedding(d_model)
        
    def forward(self, x: torch.LongTensor, mask=None) -> torch.FloatTensor:
        return self.word_embedding(x) + self.position_embedding(x)

In [37]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()
encoder(emb(torch.randint(1000, (5, 30))))

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of signle head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of signle head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[

tensor([[[-1.6346e-01,  1.2155e-01, -1.3424e+00,  ...,  8.9453e+00,
          -9.3236e-01,  4.7319e-02],
         [ 1.4943e+00, -2.8081e+00, -2.6555e+00,  ...,  8.0715e+00,
          -3.1153e+00, -3.5599e+00],
         [ 1.4497e+00, -3.9774e+00, -3.1722e+00,  ...,  7.6725e+00,
           5.8214e-01,  2.7863e+00],
         ...,
         [-3.5097e+00, -1.9654e+00, -2.6983e+00,  ...,  9.6948e+00,
          -1.0771e+00,  2.2490e+00],
         [-1.3410e+00, -3.8619e+00, -3.9282e+00,  ...,  1.0554e+01,
          -1.1946e+00,  1.8432e+00],
         [-8.7771e-01, -2.0514e+00, -3.6552e+00,  ...,  9.7941e+00,
          -2.6640e+00,  2.7551e+00]],

        [[-8.1897e-02, -5.5409e+00, -3.5413e+00,  ...,  6.6499e+00,
           4.8690e-01,  3.5351e+00],
         [ 3.9006e+00, -3.7371e-01, -6.2669e-01,  ...,  5.7385e+00,
           3.2112e+00,  3.2876e+00],
         [ 2.4043e-01,  6.7510e-01, -5.3848e-01,  ...,  9.6645e+00,
          -6.3463e-01,  2.2420e+00],
         ...,
         [-1.0896e+00, -2


# Putting it All Together

Let's put everything together now.


In [38]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.enc_dec_block)

In [39]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()
decoder = TransformerDecoder()

In [40]:
src_ids = torch.randint(1000, (5, 30))
tgt_ids = torch.randint(1000, (5, 30))
x = encoder(emb(src_ids))
decoder(emb(tgt_ids), x)

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=t

tensor([[[  4.1734,  -1.1576,  -9.5102,  ...,   1.2630,   1.9948,   2.2728],
         [  5.4421,   3.6629,  -3.3422,  ...,  -0.3909,  -0.8506,   3.4127],
         [  5.3183,   1.8572, -11.6261,  ...,  -0.3822,  -2.3306,   4.0876],
         ...,
         [  4.2614,  -0.8675, -12.0535,  ...,   1.8744,   0.1623,   8.5918],
         [ -1.5347,  -1.1825,  -8.7053,  ...,  -0.0344,  -1.1608,   2.3443],
         [  3.7099,   0.3772,  -8.5302,  ...,   5.5246,  -1.0878,   5.6007]],

        [[  3.0506,   3.3710,  -9.0028,  ...,   3.7933,   2.6592,  -1.5478],
         [  2.2161,  -1.3730,  -5.7518,  ...,   3.8405,   0.6954,   5.6836],
         [  2.3288,  -0.2974,  -9.2382,  ...,   3.4638,   0.8501,   7.2387],
         ...,
         [  3.4776,  -3.0073,  -5.2200,  ...,   3.3558,   1.6917,   1.0592],
         [  1.7754,  -2.0260,  -6.0141,  ...,   4.4846,   1.0798,   5.5728],
         [  1.6747,   1.0592, -14.2253,  ...,   3.2874,  -1.5582,   7.7551]],

        [[ -1.0860,  -2.2918,  -8.1923,  ...